# How to Use Eir Spatial Models
The static spatial models that Eir offers are the Hub and Strong Infectious models. This tutorial will use the Hub model as an example, but the same principle applies for the Strong Infectous model. The details about the model can be found in Fujie & Odagaki's paper, cited in the Eir JOSS paper. You could also find details in the docs/ folder. 

In [2]:
from Eir import HubSIR, HubSIRV # import the relevant Hub models that will be used in tutorial
import numpy as np # standard imports
import pandas as pd
from matplotlib import pyplot as plt

## Synopsis of how these spatial models work
Assume there are N separate individuals in a locality, confined to an LxL grid, with each person's location within the grid being randomly generated. These individuals do not change position, hence the "static" spatial model. 

To become infected, or leave the S compartment, we use a formula pulled from Fujie & Odagaki's paper. This formula is based on 3 variables: *r*, which represents the distance between the infectious person and susceptible person, *r0*, which is considered the "spreading radius" of the infectious person, where outside this radius, there is a 0% chance that the infectious person will spread it to the susceptible person outside the radius, and *w <sub>0* indicates the probability of an infectious person infecting a susceptible person 0 units away. It is better to simply look at the formula used in their article, as I still cannot figure out Markdown formatting. 

In the cell below, we initialize Hub model specific parameters that are used when generating the object.
   

In [4]:
pss = .17 # the probability of being a super spreader; has different definitions for Hub and Strong Infectious model. The differences of how super spreaders are treated can be found in documentation.
rstart = 3 # the distance of a normal spreader's spreading radius. In the Hub Model, this is scaled up by a factor of k for super spreaders.
side = 100 # the size of one side of the square plane that the individuals are confined to.
w0 = .93 # the w0 used in the formula; the probability an infectious person infects a susceptible person 0 units away
k = 6 ** 0.5 # the Hub constant that is used for scaling up rstart for super spreaders in the Hub model
alpha = 2 # another constant used in the spreading probability formula
days = 31 # the number of days being simulated

 

In the next cell, the parameters specific to the SIR model are initialized.

In [12]:
S0 = 9999 # initial number of susceptible individuals
I0 = 1 # initial number of infecteds
R0 = 0 # initial number of recovereds
gamma = .15 # the probability that an individual moves from I -> R. 

All the parameters have been initialized. The next step is to create a HubSIR object.

In [13]:
sim = HubSIR(pss=pss, rstart=rstart, side=side, w0=w0, hubConstant=k, alpha=alpha, days=days, S0=S0, I0=I0, R0=R0, gamma=gamma) # initialize the HubSIR object

When you run a simulation for a particular object, you use the run() method. This method has a default paremeter getDetails which is initialized to True. If getDetails=True, then the run() method returns a Simul_Details object, which can be used to delve deeper into the simulation. It will be featured later in this tutorial.

In [14]:
details = sim.run() # run the simulation and return a Simul_Details object

The simulation can also be retrieved as a pandas dataframe using the toDataFrame() method. It is important to do this only after the run() method has been called.

In [15]:
df = sim.toDataFrame()
df

,Days,Susceptible,Infected,Recovered
0,0.0,9999.0,1.0,0.0
1,1.0,9994.0,6.0,0.0
2,2.0,9934.0,65.0,1.0
3,3.0,9804.0,182.0,14.0
4,4.0,9668.0,281.0,51.0
5,5.0,9487.0,416.0,97.0
6,6.0,9266.0,567.0,167.0
7,7.0,9012.0,738.0,250.0
8,8.0,8708.0,926.0,366.0
9,9.0,8335.0,1143.0,522.0


## Simul_Details object

Now, it is time to learn how to interact with the Simul_Details object. This object allows you to see a variety of things, from a particular person's state history to transmission chains. One thing you might want to do is find the state history of a particular person's number, such as the 4th person. You can simply use the personHistory() method and pass in the number person you want to view

In [17]:
stateOfNine = details.personHistory(9) # get the state history of person 9. Important to note that the people's numbers go from person 0, 1, 2... N-1, where N is the starting population of the simulation.
stateOfNine

[(0, 'S'), (14, 'I')]

From the above cell, we can see that for Person 9, they were susceptible on the 0th day. Then, on the 14th day, they changed to be infected. As you can see, it is very easy to read which state a given person was on what day and how they changed as the simulation progressed. 

Let's say that we also wanted to find the tranmission history of person 9; that is, who did they infect and on what day did they infect that person. For this, we'd use the personTransmissionHistory function and input the person number as an argument. 

In [18]:
transOfNine = details.personTransmissionHistory(9) # get the transmission history of person 9
transOfNine

[(5265, 15)]

The way to interpret the above output is that person 9 infected person 5265 on day 15. This list can be far more extensive and have multiple tuples, as can been in the following output:

In [29]:
transOfSeven = details.personTransmissionHistory(7) # get the transmission history of person 7
transOfSeven

[(328, 16),
 (420, 16),
 (519, 16),
 (816, 16),
 (2245, 16),
 (5663, 16),
 (6117, 16),
 (6431, 16),
 (7006, 16),
 (7030, 16),
 (7157, 16),
 (7469, 16),
 (8154, 16),
 (8290, 16),
 (8461, 16),
 (8549, 16),
 (9291, 16),
 (8215, 17)]

You can also have an empty list, which means they didn't transmit the disease to anyone. 

If you want to get ALL of the transmissions of a particular day, then you can use the transmissionHistoryOnDay method and input the day as an argument.

In [40]:
transmission = details.transmissionHistoryOnDay(3) # get the entire transmission history on day 3. The days go from [1, days of simultion]
transmission 

[(84, 667),
 (84, 2187),
 (84, 2531),
 (84, 2595),
 (84, 2694),
 (84, 4228),
 (84, 4255),
 (84, 4710),
 (84, 5124),
 (84, 5172),
 (84, 5315),
 (84, 5683),
 (84, 5751),
 (84, 6152),
 (84, 6293),
 (84, 7560),
 (84, 8560),
 (84, 8573),
 (84, 9644),
 (84, 9804),
 (400, 7301),
 (427, 162),
 (427, 5043),
 (506, 895),
 (1087, 1376),
 (1087, 6011),
 (1087, 8498),
 (1087, 9822),
 (1522, 1309),
 (1534, 8593),
 (2118, 7082),
 (2118, 8986),
 (2190, 226),
 (2190, 637),
 (2190, 1048),
 (2190, 1256),
 (2190, 2967),
 (2190, 6490),
 (2190, 6838),
 (2190, 7246),
 (2190, 7609),
 (2190, 8365),
 (2285, 132),
 (2285, 2421),
 (2285, 5943),
 (2285, 6548),
 (2285, 7086),
 (2285, 7275),
 (2285, 8233),
 (2285, 8563),
 (2285, 8636),
 (2379, 6928),
 (2840, 2903),
 (2840, 6023),
 (2840, 7958),
 (3315, 864),
 (3315, 1476),
 (3315, 2336),
 (3315, 3024),
 (3315, 3344),
 (3315, 5627),
 (3315, 6430),
 (3315, 7897),
 (3315, 8124),
 (3315, 8288),
 (3315, 8366),
 (3315, 8527),
 (3315, 9282),
 (3315, 9726),
 (3651, 1081),
 

Each tuple represents a transmission event. The way to interpret it is the first numbered person infected the second number person. For example, for the first tuple in the list, person 84 infected person 667 on day 3. 

Finally, if you want to examine how many transmissions every person has throughout the duration of the simulation, you can use the sortedTranmissions() method. This method returns a list of tuples that will return the number of tranmissions each person has, sorted from highest number of transmissions to lowest. 

In [43]:
transmissions_sorted = details.sortedTransmissions() # get the sorted transmissions
transmissions_sorted

[(32.0, [642]),
 (28.0, [576]),
 (26.0, [1905, 3805, 6187]),
 (25.0, [120]),
 (24.0, [137, 775, 2165]),
 (23.0, [52, 84, 563, 886, 3152]),
 (22.0, [447, 496, 554, 1046, 1263]),
 (21.0, [1076, 5880, 9242]),
 (20.0, [214, 4053, 6738]),
 (19.0, [762, 1214, 1406, 1440, 1716, 3219]),
 (18.0, [7, 398, 1025, 1448, 2161, 2447, 3204, 3315, 3749, 8148]),
 (17.0, [50, 207, 549, 1148, 1340, 1692, 3538, 4243, 4862, 9619]),
 (16.0, [181, 340, 372, 708, 1813, 2390, 3685, 4139, 4176, 4567, 5611]),
 (15.0, [2, 217, 350, 1120, 1247, 1349, 1371, 1545, 1817, 1843, 2098, 5760]),
 (14.0,
  [58,
   172,
   428,
   543,
   658,
   793,
   891,
   950,
   1110,
   1482,
   1553,
   1641,
   2017,
   2061,
   2102,
   2443,
   2708,
   4311,
   4777,
   5368,
   5677,
   6118,
   6283,
   6620,
   8373,
   8976]),
 (13.0,
  [142,
   291,
   374,
   471,
   787,
   882,
   1974,
   2304,
   2647,
   3626,
   3671,
   3965,
   4543,
   4629,
   4825,
   4866,
   7008,
   7302,
   7807,
   9185]),
 (12.0,
  [653,


The way to interpret each tuple is as follows: the 0th element of the tuple will be an integer representing the number of transmissions made. The 1st element will be a list of the numbers of the people who had that many transmissions. For example, for the 0th tuple in the list, 32 transmissions were made by person 642. But for the 2nd indexed tuple, there were 26 transmissions made by person 1905, 3805, and 6187. 